In [43]:
import cv2
import numpy as np
import win32api,win32con,win32gui,win32ui
from PIL import Image,ImageGrab

In [44]:
l=[(328,630),(329,852),(329,1074),(386,667),(346,1022),(387,1399),(514,333),(600,615),(540,960),(600,1362),(516,1700)]
w=[32,32,32,72,63,72,100,180,106,190,100]

In [45]:
class box:
    def __init__(self,center,len):
        self.xywh=[center[0]-len//2,center[1]-len//2,len,len]
        self.len=len
        self.state=0
        self.shape=(len,len)
        self.center=center
    def get_self_img(self,img):
        self_img=img[self.xywh[0]:self.xywh[0]+self.len,self.xywh[1]:self.xywh[1]+self.len]
        return self_img
    def update_state(self,img):
        self_img=self.get_self_img(img)
        hsv = cv2.cvtColor(self_img, cv2.COLOR_BGR2HSV)
        # 颜色识别(红色)，过滤红色区域
        lower_red1 = np.array([0, 43, 46])  # 红色阈值下界
        higher_red1 = np.array([10, 255, 255])  # 红色阈值上界
        mask_red1 = cv2.inRange(hsv, lower_red1, higher_red1)
        lower_red2 = np.array([156, 43, 46])  # 红色阈值下界
        higher_red2 = np.array([180, 255, 255])  # 红色阈值上界
        mask_red2 = cv2.inRange(hsv, lower_red2, higher_red2)
        mask_red = cv2.add(mask_red1, mask_red2)  # 拼接过滤后的mask
        pd_color=np.mean(mask_red)
        if pd_color<50: #蓝色
            self.state=1
        else: #红色
            self.state=0

In [46]:
box_list=[]
for i in range(11):
    box_list.append(box(l[i],w[i]))

In [47]:
def window_capture():
    hwnd = 0  # 窗口的编号，0号表示当前活跃窗口
    # 根据窗口句柄获取窗口的设备上下文DC（Divice Context）
    hwndDC = win32gui.GetWindowDC(hwnd)
    # 根据窗口的DC获取mfcDC
    mfcDC = win32ui.CreateDCFromHandle(hwndDC)
    # mfcDC创建可兼容的DC
    saveDC = mfcDC.CreateCompatibleDC()
    # 创建bigmap准备保存图片
    saveBitMap = win32ui.CreateBitmap()
    # 获取监控器信息
    MoniterDev = win32api.EnumDisplayMonitors(None, None)
    w = 1920
    h = 1080
    # print w,h　　　#图片大小
    # 为bitmap开辟空间
    saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # 高度saveDC，将截图保存到saveBitmap中
    saveDC.SelectObject(saveBitMap)
    # 截取从左上角（0，0）长宽为（w，h）的图片
    saveDC.BitBlt((0, 0), (w, h), mfcDC, (640, 220), win32con.SRCCOPY)
    #saveBitMap.SaveBitmapFile(saveDC, filename)
    bmpinfo = saveBitMap.GetInfo()
    bmpstr = saveBitMap.GetBitmapBits(True)
    ###生成图像
    im_PIL = Image.frombuffer('RGB', (bmpinfo['bmWidth'], bmpinfo['bmHeight']), bmpstr, 'raw', 'BGRX', 0, 1)
    return im_PIL

In [49]:
import time
while 1:
    time.sleep(2)
    #img=window_capture()
    #img=np.array(img)[:,:,::-1]
    img=cv2.imread('imgs/start.png')
    for i in range(11):
        box_list[i].update_state(img)
        if box_list[i].state==1:
            print(f'第{i}个是蓝色')
            mouse_move=(box_list[i].center[0]-960, box_list[i].center[1]-540)
            win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, mouse_move[0], mouse_move[1])
            #win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0)
            #win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0)
            win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, -mouse_move[0], -mouse_move[1])
            time.sleep(0.5)
            #win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, 960, 450)
    

第8个是蓝色
第8个是蓝色
第8个是蓝色
第8个是蓝色
第8个是蓝色
第8个是蓝色
第8个是蓝色
第8个是蓝色
第8个是蓝色
第8个是蓝色


KeyboardInterrupt: 